# Introduction
---
The purpose of this notebook is to generate and compute the 1-edit distance between all the possible rooted and unlabeled trees given a fixed `depth` and `maximum_degree`.

## Notebook configuration
Configure several notebook configuration settings.

In [4]:
# Disable some warnings

import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last_expr'

# Use full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

## Libraries / Packages
Import several useful packages for the notebook and configure some extra options.

In [5]:
# Miscellaneous libraries
import time
import random
import itertools
import numpy as np
import pandas as pd
import networkx as nx

from tqdm.notebook import tqdm
from collections import defaultdict, Counter

# Setup some options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100


# Visualization
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.style.use('seaborn-whitegrid')

# Generating all multisets
---
An efficient way of storing representations of rooted unlabel trees of fixed depth and maximum degree is using multisets. Let's generate all multisets;

In [40]:
###
# Some utility functions

def pad(arr, n):
    '''
    Auxiliary function that pads an array up to a given length.
    
    Parameters:
        - arr: (array_like) Array to pad.
        - n: (int) Desired length of the padded array.
        
    Returns:
        - (np.array<int>) Padded array.
    '''
    padded = np.zeros(n)
    padded[:len(arr)] = arr
    return padded.astype(int)


def generatePermutations(arr):
    '''
    Auxiliary function that generates all unique permutations of a given array.
    
    Parameters:
        - arr: (array_like) Array to generate permutations from.
        
    Returns:
        - (list<np.array>) List with all the generated permutations.
    '''
    permutations = set(list(itertools.permutations(arr)))
    permutations = [np.array(permutation).astype(int) for permutation in permutations]
    return permutations


def findCombinations(arr, index, num, reducedNum):
    '''
    Function that generates all multisets that sum up to a given value.
    
    Parameters:
        - arr: (array_like) Array to store the combination
        - index: (int) Next location in array
        - num: (int) Given number
        - reducedNum: (int) Reduced number
        
    Returns:
        - None
    '''
    # Base condition
    if (reducedNum < 0):
        return
    # If combination is found, append it to results
    if (reducedNum == 0):
        all_combinations.append([arr[i] for i in range(index)])
        return
    # Find the previous number stored in arr[]
    prev = 1 if(index == 0) else arr[index - 1]
    # Note loop starts from previous number i.e. at array location index - 1
    for k in range(prev, num + 1):
        # Next element of array is k
        arr[index] = k
        # Call recursively with reduced number
        findCombinations(arr, index + 1, num, reducedNum - k)
        
###
# Generate all multisets (hence trees of depth 2)
MAX_DEGREE = 7
ALL_MULTISETS = []
        
for degree in range(MAX_DEGREE + 1):
    all_combinations = []
    findCombinations([0] * (MAX_DEGREE + 1), 0, degree, degree)
    # Pad the found combinations
    all_combinations = [pad(x, n=MAX_DEGREE + 1) for x in all_combinations]
    # Generate all the unique permutations and flatten the output
    all_permutations = [generatePermutations(x) for x in all_combinations]
    all_permutations = [item for sublist in all_permutations for item in sublist]
    ALL_MULTISETS.append(all_permutations)

# Flatten the list of all trees
ALL_MULTISETS = [item for sublist in ALL_MULTISETS for item in sublist]

len(ALL_MULTISETS), ALL_MULTISETS




(6435,
 [array([0, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 0, 1, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 1, 0]),
  array([0, 0, 0, 1, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 1, 0, 0]),
  array([1, 0, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 1, 0, 0, 0, 0, 0, 0]),
  array([1, 0, 0, 0, 1, 0, 0, 0]),
  array([0, 0, 0, 0, 1, 0, 1, 0]),
  array([0, 0, 0, 0, 0, 0, 1, 1]),
  array([0, 0, 0, 0, 0, 1, 1, 0]),
  array([1, 0, 0, 0, 0, 0, 1, 0]),
  array([0, 1, 0, 0, 0, 1, 0, 0]),
  array([0, 0, 0, 0, 1, 0, 0, 1]),
  array([0, 0, 1, 0, 0, 1, 0, 0]),
  array([0, 1, 0, 0, 1, 0, 0, 0]),
  array([0, 0, 0, 0, 0, 1, 0, 1]),
  array([1, 0, 0, 0, 0, 0, 0, 1]),
  array([0, 0, 0, 1, 0, 1, 0, 0]),
  array([1, 0, 0, 1, 0, 0, 0, 0]),
  array([0, 0, 1, 0, 1, 0, 0, 0]),
  array([1, 0, 1, 0, 0, 0, 0, 0]),
  array([0, 0, 0, 1, 1, 0, 0, 0]),
  array([0, 1, 0, 0, 0, 0, 1, 0]),
  array([1, 1, 0, 0, 0, 0, 0, 0]),
  array([0, 0, 1, 0, 0, 0, 1, 0]),
  array([0, 0

### Compute adjacency between multisets
---
We will attempt to use elementary operations to determine if two multisets are at 1-edit operation between two trees.

In [38]:
def computeMultisetAdjacency(multiset1, multiset2):
    '''
    
    '''
    diff = multiset1 - multiset2
    diff_ = np.where(diff)[0]
    # If the summation of diff is 0, at least they must differ in 2 positions
    if np.sum(diff) == 0:
        # Make sure that they differ ONLY in 2 positions and that they are adjacent
        if len(diff_) == 2 and np.absolute(diff_[0] - diff_[1]) == 1:
            return True
    # Handle the edge case specially (i.e. adding/deleting single leaves)
    if np.absolute(np.sum(diff)) == 1:
        if len(diff_) == 1 and diff_[0] == 0:
            return True
    return False


# Compute the adjacency matrix between multisets
n = len(ALL_MULTISETS)
A = np.zeros((n, n))

for i in range(n):
    for j in range(i, n):
        A[i, j] = computeMultisetAdjacency(ALL_MULTISETS[i], ALL_MULTISETS[j])
        A[j, i] = A[i, j]

print(ALL_MULTISETS), print(A)



[array([0, 0, 0, 0]), array([0, 0, 1, 0]), array([1, 0, 0, 0]), array([0, 0, 0, 1]), array([0, 1, 0, 0]), array([1, 0, 1, 0]), array([1, 1, 0, 0]), array([1, 0, 0, 1]), array([0, 1, 1, 0]), array([0, 1, 0, 1]), array([0, 0, 1, 1]), array([0, 2, 0, 0]), array([0, 0, 0, 2]), array([2, 0, 0, 0]), array([0, 0, 2, 0]), array([1, 1, 0, 1]), array([0, 1, 1, 1]), array([1, 1, 1, 0]), array([1, 0, 1, 1]), array([0, 2, 0, 1]), array([0, 0, 2, 1]), array([0, 1, 2, 0]), array([0, 2, 1, 0]), array([2, 0, 0, 1]), array([1, 0, 2, 0]), array([1, 2, 0, 0]), array([2, 1, 0, 0]), array([2, 0, 1, 0]), array([0, 0, 1, 2]), array([1, 0, 0, 2]), array([0, 1, 0, 2]), array([0, 0, 0, 3]), array([0, 0, 3, 0]), array([0, 3, 0, 0]), array([3, 0, 0, 0])]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


(None, None)

### Unordered sampling with replacement
---
$\binom{n + k - 1}{k} = \frac{(n + k - 1)!}{k!((n + k - 1) - k)!},$
where $n$ is the number of possible elements and $k$ is the number of samples to take.

In [41]:
def binomial(n, k):
    '''Auxiliary function for computing binomial numbers.'''
    if not 0 <= k <= n:
        return 0
    b = 1
    for t in range(min(k, n - k)):
        b *= n
        b //= t + 1
        n -= 1
    return b

n = len(ALL_MULTISETS)
k = MAX_DEGREE

f'{binomial(n + k - 1, k):,}'





'90,954,904,732,217,610,881,940'